In [3]:
%pylab inline
import os
import scipy.io
import numpy as np
from keras.models import Sequential
from keras.layers import Convolution2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adam



Populating the interactive namespace from numpy and matplotlib


In [4]:
######################### load file - training data ####################

#Go to the data directory 
os.chdir('C:\Liliana\calcium test 29 9V')

training_data = scipy.io.loadmat('trigg_cell_test29_9V_9900.mat')

#Define variables 
N_train_cell = len(training_data['cell_classified'][0])
Int_profile_len = len(training_data['cell_classified'][0,0]['intensity'][0])

#Matrix for intensity values
train_dt = np.zeros((N_train_cell,Int_profile_len))
for i in range(0,N_train_cell):
    train_dt[:,:]= training_data['cell_classified'][0,i]['intensity']

#Array for the classes 
train_cl = training_data['cell_classified'][0]['triggering'].astype(int)

X = train_dt#np.random.rand(10000, 128).astype("float32")
y = train_cl #np.random.randint(3, size=(10000,1))


# process the data to fit in a keras CNN properly
# input data needs to be (N, C, X, Y) - shaped where
# N - number of samples
# C - number of channels per sample
# (X, Y) - sample size

X = X.reshape((N_train_cell, Int_profile_len,1, 1))
y = np_utils.to_categorical(y)
#print(y)



In [5]:
######################### load file - testing data ####################

#Go to the data directory 
os.chdir('C:\Liliana\calcium test 58 9V')

test_data = scipy.io.loadmat('trigg_cell_test58_9V_9900.mat')

#Define variables 
N_test_cell = len(training_data['cell_classified'][0])
Int_profile_len_test = len(training_data['cell_classified'][0,0]['intensity'][0])

#Matrix for intensity values
test_dt = np.zeros((N_test_cell,Int_profile_len_test))
for i in range(0,N_test_cell):
    test_dt[:,:]= test_data['cell_classified'][0,i]['intensity']

#Array for the classes 
test_cl = test_data['cell_classified'][0]['triggering'].astype(int)



In [6]:
# define a CNN
# see http://keras.io for API reference

cnn = Sequential()
cnn.add(Convolution2D(64, 3, 1, border_mode="same",activation="relu",input_shape=(Int_profile_len,1, 1)))
cnn.add(Convolution2D(64, 3, 1, border_mode="same", activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2,1)))

#border_mode = same => output size is the same of input size. The filter does not go outside the bounds of the input
#ReLU => thresholding at zero. All the negative values will be = 0, values bigger than zero will mantain the same value.
cnn.add(Convolution2D(128, 3, 1, border_mode="same", activation="relu"))
cnn.add(Convolution2D(128, 3, 1, border_mode="same", activation="relu"))
cnn.add(Convolution2D(128, 3, 1, border_mode="same", activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2,1)))
    
cnn.add(Convolution2D(256, 3, 1, border_mode="same", activation="relu"))
cnn.add(Convolution2D(256, 3, 1, border_mode="same", activation="relu"))
cnn.add(Convolution2D(256, 3, 1, border_mode="same", activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2,1)))
    
cnn.add(Flatten())
cnn.add(Dense(1024, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(5, activation="softmax"))

# define optimizer and objective, compile cnn

cnn.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0))

# train

cnn.fit(X, y, nb_epoch=20, verbose = 2)

C:\Users\Liliana\Anaconda2\envs\myenv2\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 1), activation="relu", input_shape=(1000, 1, ..., padding="same")`
C:\Users\Liliana\Anaconda2\envs\myenv2\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 1), activation="relu", padding="same")`
C:\Users\Liliana\Anaconda2\envs\myenv2\lib\site-packages\ipykernel\__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 1), activation="relu", padding="same")`
C:\Users\Liliana\Anaconda2\envs\myenv2\lib\site-packages\ipykernel\__main__.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 1), activation="relu", padding="same")`
C:\Users\Liliana\Anaconda2\envs\myenv2\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 1), activation="relu",

Epoch 1/20


KeyboardInterrupt: 

In [ ]:
#Evaluation code which makes prediction
out = cnn.predict(test_dt.reshape((N_train_cell, Int_profile_len,1, 1)))

In [ ]:
%pylab inline
print(out[0])
print(test_cl[0])

print(float(np.sum(np.argmax(out,1)==test_cl))/float(test_cl.shape[0])*100.0)

In [ ]:
for i in range(0,4):
    plot(test_dt[i,:])
    print(test_cl[i])

In [ ]:
plot(out[:40,0])
plot(out[:40,1])
plot(out[:40,2])